<a href="https://colab.research.google.com/github/TarnishingColors/web_mining_hate-speech-detection/blob/main/web_mining_project_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving d.csv to d.csv
Saving d1.csv to d1.csv


In [2]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['d.csv']), encoding="utf-8", encoding_errors='ignore')

In [3]:
df1 = pd.read_csv(io.BytesIO(uploaded['d1.csv']), encoding="utf-8", encoding_errors='ignore')

In [4]:
d1_df = pd.DataFrame()
d1_df['class'] = df1['label']
d1_df['tweet'] = df1['tweet']
d1_df

,class,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [5]:
!pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.6 MB/s eta 0:00:00


In [6]:
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

In [7]:
#custom preprocessing for this specific dataset
df.drop(['count', 'offensive_language','neither','hate_speech'], axis = 1, inplace = True)
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_first = df["class"].transform(lambda x: x.replace([1,2], "XX"))
df["class"] = df_first.transform(lambda x: x.replace(0,1).replace("XX",0)) #hate-speach = 1 in class everitying else  = 0
display(df)

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,0,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,0,"you've gone and broke the wrong heart baby, an..."
24780,0,young buck wanna eat!!.. dat nigguh like I ain...
24781,0,youu got wild bitches tellin you lies


In [8]:
df = df.append(d1_df)
df

<ipython-input-8-e0a1b7225b8f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d1_df)


,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [9]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00


In [105]:
import html
import re

import nltk
import pandas as pd
import torch
from emot.emo_unicode import UNICODE_EMOJI  # For emojis
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator

from toolz import pipe
from functools import reduce


def compose(*functions):
    return reduce(lambda f, g: lambda x: f(g(x)), functions[::-1])


# class for preprocessing
class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, func: callable):
        self.func = func

    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self

    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        _df = df.copy()
        _df["tweet"] = _df["tweet"].transform(lambda x: self.func(x))
        return _df


# Lemmatization
nltk.download('stopwords')
nltk.download('punkt')

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


class PrepFuncs:
    re_username = r"@\w+"
    re_exclamation = ""
    re_link = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                  '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    re_all_words = "[^a-z ]+"

    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()


    @classmethod
    def get_wordnet_pos(self, treebank_tag: str):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    # Function for converting emojis into word
    @classmethod
    def convert_emojis(self, text: str):
        text = html.unescape(text)
        for emot in UNICODE_EMOJI:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
        return text

    @classmethod
    def lemmatize(self, text: str):
        lemmatized = [
            self.lemmatizer.lemmatize(word, self.get_wordnet_pos(pos_tag)) 
            for (word, pos_tag) in nltk.pos_tag(nltk.word_tokenize(text))
            ]
        return " ".join(lemmatized)

    @classmethod
    def remove_stopwords(self, tweet: str):
        tokens_without_sw = " ".join(
            [word for word in word_tokenize(tweet) if not word in [*self.stop_words, 'rt']]
        )
        return tokens_without_sw
    
    @classmethod
    def delete_usernames(self, text: str):
        return re.sub(self.re_username, "username", text)
    
    @classmethod
    def delete_links(self, text: str):
        return re.sub(self.re_link, "link", text)
    
    @classmethod
    def delete_special_symbols(self, text: str):
        return re.sub(self.re_all_words, " ", text)
    
    @classmethod
    def to_lower(self, text: str):
        return text.lower()
    

    @staticmethod
    def preprocessing():
        return compose(
            PrepFuncs.delete_usernames,
            PrepFuncs.delete_links,
            PrepFuncs.convert_emojis,
            PrepFuncs.to_lower,
            PrepFuncs.delete_special_symbols,
            PrepFuncs.lemmatize,
            PrepFuncs.remove_stopwords,
        )


def preprocessing(text: str):
    return PrepFuncs.preprocessing()(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from transformers import BertTokenizer, BertModel


# Specify the pre-trained model name.
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'  # uncased means this tokenizer will also lower-case automatically

# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


class Vectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer: callable, max_len: int = 32):
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = tokenizer

        self.config = {
            'add_special_tokens': True,
            'padding': 'max_length',
            'max_length': max_len,
            'truncation': True,
            'pad_to_max_length': True,
            'return_attention_mask': False,
            'return_tensors': 'pt',
        }

    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self

    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        with torch.no_grad():
            _df = df \
                .apply(lambda x: self.tokenizer(x['tweet'], **self.config)['input_ids'], axis=1) \
                .apply(lambda x: pd.Series(self.model(x)[0].flatten()))

        return _df

In [36]:
!pip install better_profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.5 MB/s eta 0:00:00


In [40]:
df.head()

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [51]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
# Example data
row = "I love this product! It's amazing!"
row2 = "Fuck this product! It's shitting arsehole bitch-ass!"

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Calculate the sentiment score
sentiment_score = sia.polarity_scores(row2)["compound"]

print(sentiment_score)

-0.7835


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [124]:
import html
from statistics import mean

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from better_profanity import profanity
from nltk.sentiment import SentimentIntensityAnalyzer


nltk.download('vader_lexicon')


class FeatureGeneration:
    excl_pattern = r'^!+$'
    sia = SentimentIntensityAnalyzer()

    @classmethod
    def _basic_row_cleaning(self, text: str):
        split_text = text.split()
        if re.match(self.excl_pattern, split_text[0]):
            split_text = split_text[1:]
        return " ".join([word for word in split_text if not (re.match(PrepFuncs.re_username, word) or word == "RT")])

    @classmethod
    def _annotate_rt(self, text: str):
        split_text = text.split()
        return 1 if len(split_text) > 0 and 'RT' in split_text else 0
    
    @classmethod
    def _contains_profanity_words(self, text: str):
        return profanity.contains_profanity(text)
    
    @classmethod
    def _sentiment_score_of_row(self, text: str):
        return self.sia.polarity_scores(text)['compound']

    @classmethod
    def _number_of_words_in_row(self, text: str):
        return len(text.split())
    
    @classmethod
    def _average_len_of_words(self, text: str):
        return mean([len(word) for word in text.split()])
    
    @classmethod
    def _number_of_commas(self, text: str):
        return text.count(",")
    
    @classmethod
    def _number_of_excl_points(self, text: str):
        return text.count("!")
    
    @classmethod
    def _number_of_question_marks(self, text: str):
        return text.count("?")
    
    @classmethod
    def _number_of_full_stops(self, text: str):
        return text.count(".")
    
    @classmethod
    def _combine_all_the_features(self, X):
        rt_flag = X.apply(lambda row: self._number_of_full_stops(row['tweet']), axis=1)
        cleaned = X.apply(lambda row: self._basic_row_cleaning(row['tweet']), axis=1)
        contains_profanity_words_flag = cleaned.apply(self._contains_profanity_words)
        sentiment_score_of_row = cleaned.apply(self._sentiment_score_of_row)
        number_of_words_in_row = cleaned.apply(self._number_of_words_in_row)
        number_of_commas = cleaned.apply(self._number_of_commas)
        number_of_excl_points = cleaned.apply(self._number_of_excl_points)
        number_of_question_marks = cleaned.apply(self._number_of_question_marks)
        number_of_full_stops = cleaned.apply(self._number_of_full_stops)

        new_columns = {
            'rt_flag': rt_flag, 
            'contains_profanity_words_flag': contains_profanity_words_flag, 
            'sentiment_score_of_row': sentiment_score_of_row, 
            'number_of_words_in_row': number_of_words_in_row,
            'number_of_commas': number_of_commas,
            'number_of_excl_points': number_of_excl_points,
            'number_of_question_marks': number_of_question_marks,
            'number_of_full_stops': number_of_full_stops
        }

        return pd.DataFrame(new_columns)

    @classmethod
    def generate_features(self, input_df: pd.DataFrame):
        column_transformer = ColumnTransformer(
            transformers=[
                ('feature_generator', FunctionTransformer(self._combine_all_the_features), ['tweet']),
            ],
        )
        return pd.DataFrame(column_transformer.fit_transform(input_df), columns=[
            'rt_flag', 
            'contains_profanity_words_flag', 
            'sentiment_score_of_row', 
            'number_of_words_in_row',
            'number_of_commas',
            'number_of_excl_points',
            'number_of_question_marks',
            'number_of_full_stops'
            ])

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [125]:
sliced_df = df.copy()[:500]

fg = FeatureGeneration()
fg.generate_features(sliced_df)

,rt_flag,contains_profanity_words_flag,sentiment_score_of_row,number_of_words_in_row,number_of_commas,number_of_excl_points,number_of_question_marks,number_of_full_stops
0,4,False,0.2755,22,0,0,0,4
1,3,False,-0.6229,13,0,2,0,3
2,0,True,-0.955,16,0,4,1,0
3,0,False,0.3612,5,0,0,0,0
4,0,True,-0.6808,23,0,0,0,0
...,...,...,...,...,...,...,...,...
495,1,True,-0.5994,15,0,0,0,1
496,0,False,0.0,6,0,0,0,0
497,4,True,0.3987,26,0,2,0,4
498,0,True,-0.2396,15,1,0,0,0


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer


# class for vectorizing
class TfIdfVectorizer():
    def __init__(self, vectorizer: callable):
        self.vectorizer = vectorizer
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        X = self.vectorizer.fit_transform(df["tweet"])
        new_df = pd.DataFrame(X.toarray(), columns=self.vectorizer.get_feature_names_out())
        new_df["rt_flag"] = df["rt_flag"]
        new_df["label"] = df["class"] # 'class' is the name of label in d.csv, so we'll need to standartize all the datasets
        return new_df


vectorizer = TfidfVectorizer(norm='l2', min_df=0.05, max_df=0.5)

In [92]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    transformed_df[transformed_df.columns.difference(['label'])], transformed_df['label'], test_size=0.2, random_state=42
)
# train_masks, test_masks, _, _ = train_test_split(
#     attention_masks, input_ids, test_size=0.2, random_state=42
# )

In [107]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(class_weight={0:1,1:7})
model.fit(train_inputs, train_labels)

RandomForestClassifier(class_weight={0: 1, 1: 7})

In [108]:
from sklearn.metrics import f1_score, confusion_matrix

predictions = model.predict(test_inputs)
f1 = f1_score(test_labels, predictions)
cm = confusion_matrix(test_labels, predictions)
print("f1:", f1)
print(cm)

f1: 0.2516914749661705
[[9354   22]
 [ 531   93]]
